<a href="https://colab.research.google.com/github/pi-mishra/PNB_stock_prediction/blob/main/PNB_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/drive/MyDrive/yes_bank/PNB.NS.csv')

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-07-01,9.60,9.95,9.45,9.85,7.076192,7659365
1,2002-07-08,9.90,10.04,9.30,9.51,6.831937,6778970
2,2002-07-15,11.22,11.22,8.60,9.30,6.681076,4132105
3,2002-07-22,9.20,9.70,8.82,8.91,6.400900,2705280
4,2002-07-29,9.00,10.12,8.20,9.15,6.573315,4085415


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1095 non-null   object 
 1   Open       1095 non-null   float64
 2   High       1095 non-null   float64
 3   Low        1095 non-null   float64
 4   Close      1095 non-null   float64
 5   Adj Close  1095 non-null   float64
 6   Volume     1095 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 60.0+ KB
